# SBTi Temperature Alignment tool - Reporting
This notebook ...

#### Setting up
Imports SBTi tools, import data providers, and load the portfolio.

In [ ]:
# Only required if you didn't install the module yet
!pip install SBTi

In [1]:
%load_ext autoreload
%autoreload 2
import SBTi
from SBTi.data.excel import ExcelProvider
from SBTi.portfolio_aggregation import PortfolioAggregationMethod
from SBTi.portfolio_coverage_tvp import PortfolioCoverageTVP
from SBTi.temperature_score import TemperatureScore, Scenario, ScenarioType, EngagementType
from SBTi.target_validation import TargetProtocol
from SBTi.interfaces import ETimeFrames, EScope
from examples.utils import anonymize
%aimport -pandas
import pandas as pd

In [2]:
# Download the dummy data
import urllib.request
import os

if not os.path.isdir("data"):
    os.mkdir("data")
if not os.path.isfile("data/data_provider_example.xlsx"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/data/data_provider_example.xlsx", "data/data_provider_example.xlsx")
if not os.path.isfile("example_portfolio.csv"):
    urllib.request.urlretrieve("https://github.com/OFBDABV/SBTi/raw/master/examples/example_portfolio.csv", "example_portfolio.csv")

In [6]:
provider = ExcelProvider(path="data/data_provider_example.xlsx")
portfolio = pd.read_csv("example_portfolio.csv", encoding="iso-8859-1")
portfolio["company_isin"] = portfolio.ISIN
portfolio, provider = anonymize(portfolio, provider)
companies = [SBTi.interfaces.PortfolioCompany.parse_obj(company) for company in portfolio.to_dict(orient="records")]
portfolio_data = SBTi.utils.get_data([provider], companies)

### Compute temperature scores
Configure computation settings

In [7]:
time_frames = list(SBTi.interfaces.ETimeFrames)
scopes = [EScope.S1S2, EScope.S3, EScope.S1S2S3]
fallback_score = 3.2
model = 4
aggregation_method = PortfolioAggregationMethod.WATS
grouping = None

In [8]:
temperature_score = TemperatureScore(
    time_frames=time_frames,
    scopes=scopes,
    fallback_score=fallback_score,
    model=model,
    aggregation_method=aggregation_method,
    grouping=grouping
)

In [9]:
amended_portfolio = temperature_score.calculate(portfolio_data)
aggregated_scores = temperature_score.aggregate_scores(amended_portfolio)

### Save anonymized data for SBTi target validation
In order for the targets to be validated by SBTi, you can save your data locally. By running the anonymize function, you can replace company identifiers with meaningless substitutes.

In [ ]:
portfolio, provider = anonymize(portfolio, provider)

In order to store the portfolio and provider data locally, two options apply.
If you use the SBTi tool from a docker container, for each of the following 3 cells:
- Run the cell
- Open an Excel file
- Right-click an Excel cell and click 'paste' or select an Excel cell and press Ctrl+v
- Save the Excel file

In [ ]:
portfolio.to_clipboard(excel=True, index=False)

In [ ]:
provider.data['fundamental_data'].to_clipboard(excel=True, index=False)

In [ ]:
provider.data['target_data'].to_clipboard(excel=True, index=False)

If you run the SBTi tool locally, you:
- Specify the directory and filenames in the cell below
- Run the cell below

In [ ]:
portfolio_filename = 'C://Users//username//Documents//portfolio.xlsx'
provider_filename = 'C://Users//username//Documents//provider.xlsx'
portfolio.to_excel(portfolio_filename, index=False)
provider.data['fundamental_data'].to_excel(provider_filename, sheet_name='fundamental_data', index=False)
provider.data['target_data'].to_excel(provider_filename, sheet_name='target_data', index=False)